In [50]:
import os
os.chdir("/root/workspace/code/sc-transformer/")
from os.path import join as pj
import argparse
import sys
sys.path.append("modules")
import utils
import numpy as np
import anndata as ad
import pandas as pd
import copy
import re

In [51]:
parser = argparse.ArgumentParser()
# parser.add_argument('--tasks', type=str, nargs='+',  default=["dogma_full", "dogma_paired_full", 
#     "dogma_paired_abc", "dogma_paired_ab",  "dogma_paired_ac", "dogma_paired_bc",
#     "dogma_single_full", "dogma_single"])
parser.add_argument('--tasks', type=str, nargs='+',  default=["teadog_full", "teadog_paired_full", 
    "teadog_paired_abc", "teadog_paired_ab",  "teadog_paired_ac", "teadog_paired_bc",
    "teadog_single_full", "teadog_single"])
parser.add_argument('--method', type=str, default='midas_embed')
parser.add_argument('--mosaic', type=int, default=1)
parser.add_argument('--sota', type=int, default=1)
parser.add_argument('--experiment', type=str, default='e0')
parser.add_argument('--model', type=str, default='default')
parser.add_argument('--init_model', type=str, default='sp_00001899')
o, _ = parser.parse_known_args()  # for python interactive
# o = parser.parse_args()

In [52]:
# midas_embed results
df_batch_bio_embed = {}
if o.mosaic == 0:
    o.tasks = [o.tasks[0]]
for task in o.tasks:
    df_batch_bio_embed[task] = pd.read_excel(pj("result", "comparison", task, o.method, o.experiment, o.init_model, "metrics_batch_bio.xlsx"))
    df_batch_bio_embed[task].rename(index={0: task}, inplace=True)
df_batch_bio_embed_cat = pd.concat(df_batch_bio_embed.values(), axis=0)

df_batch_bio_embed_cat["Task"] = df_batch_bio_embed_cat.index
df_batch_bio_embed_cat.rename(index={i: "midas_embed" for i in df_batch_bio_embed_cat.index}, inplace=True)
df_batch_bio_embed_cat


,iLISI,graph_conn,kBET,NMI,ARI,il_score_f1,cLISI,Task
midas_embed,0.666115,0.955491,0.544772,0.677185,0.751375,0.699236,0.989207,teadog_full
midas_embed,0.676713,0.960705,0.438826,0.682634,0.754433,0.694833,0.988032,teadog_paired_full
midas_embed,0.666887,0.948209,0.488542,0.679672,0.752752,0.695489,0.989310,teadog_paired_abc
midas_embed,0.656811,0.960677,0.323292,0.642850,0.709505,0.687988,0.984834,teadog_paired_ab
midas_embed,0.654473,0.965107,0.472720,0.653826,0.714578,0.697737,0.988069,teadog_paired_ac
midas_embed,0.672395,0.954003,0.527172,0.676661,0.735766,0.692793,0.987881,teadog_paired_bc
midas_embed,0.626640,0.914731,0.210860,0.642631,0.703517,0.702551,0.988799,teadog_single_full
midas_embed,0.573920,0.917253,0.056939,0.292352,0.289622,0.424888,0.883512,teadog_single


In [53]:
# sota results
if o.sota == 1:
    methods = [
        "midas_feat+wnn",
        "mofa",
        "liger+wnn",
        "harmony+wnn",
        "scanorama_embed+wnn",
        "scanorama_feat+wnn",
        "bbknn",
        "seurat_rpca+wnn",
        "seurat_cca+wnn",
        "pca+wnn",
    ]

    df_sota = {}
    for method in methods:
        if "midas" in method:
            df_sota[method] = pd.read_excel(pj("result", "comparison", o.tasks[0], method, o.experiment, o.init_model, "metrics_batch_bio.xlsx"))
        else:
            df_sota[method] = pd.read_excel(pj("result", "comparison", re.sub("_transfer", "", o.tasks[0]), method, "metrics_batch_bio.xlsx"))
        df_sota[method].rename(index={0: method}, inplace=True)
    df_sota_cat = pd.concat(df_sota.values(), axis=0)

    df_sota_cat[["Task"]] = o.tasks[0]
    df_sota_cat.loc["midas_feat+wnn", "Task"] = o.tasks[0]
    df_sota_cat
    df_cat = pd.concat([df_batch_bio_embed_cat, df_sota_cat], axis=0)
else:
    df_cat = df_batch_bio_embed_cat


In [54]:
df_mean_cat = copy.deepcopy(df_cat)
df_mean_cat["batch_score"] = df_cat[["iLISI", "graph_conn", "kBET"]].mean(axis=1)
df_mean_cat["bio_score"] = df_cat[["NMI", "ARI", "il_score_f1", "cLISI"]].mean(axis=1)
df_mean_cat["overall_score"] = 0.4 * df_mean_cat["batch_score"] + 0.6 * df_mean_cat["bio_score"]
df_mean_cat = df_mean_cat[["Task", "iLISI", "graph_conn", "kBET", "batch_score", "NMI", "ARI", "il_score_f1", "cLISI", "bio_score", "overall_score"]]
df_mean_cat_sorted = df_mean_cat.sort_values("overall_score", ascending=False, inplace=False)
df_mean_cat_sorted

,Task,iLISI,graph_conn,kBET,batch_score,NMI,ARI,il_score_f1,cLISI,bio_score,overall_score
midas_embed,teadog_full,6.661146e-01,0.955491,0.544772,0.722126,0.677185,0.751375,0.699236,0.989207,0.779251,0.756401
midas_embed,teadog_paired_bc,6.723954e-01,0.954003,0.527172,0.717857,0.676661,0.735766,0.692793,0.987881,0.773275,0.751108
midas_embed,teadog_paired_abc,6.668870e-01,0.948209,0.488542,0.701213,0.679672,0.752752,0.695489,0.989310,0.779306,0.748068
midas_embed,teadog_paired_full,6.767134e-01,0.960705,0.438826,0.692082,0.682634,0.754433,0.694833,0.988032,0.779983,0.744822
midas_embed,teadog_paired_ac,6.544733e-01,0.965107,0.472720,0.697433,0.653826,0.714578,0.697737,0.988069,0.763552,0.737105
midas_embed,teadog_paired_ab,6.568112e-01,0.960677,0.323292,0.646927,0.642850,0.709505,0.687988,0.984834,0.756294,0.712547
midas_feat+wnn,teadog_full,7.796789e-01,0.965997,0.511723,0.752466,0.575824,0.447405,0.659653,0.986248,0.667283,0.701356
bbknn,teadog_full,6.156643e-01,0.988402,0.230111,0.611393,0.682722,0.751187,0.760806,0.847084,0.760450,0.700827
midas_embed,teadog_single_full,6.266399e-01,0.914731,0.210860,0.584077,0.642631,0.703517,0.702551,0.988799,0.759374,0.689255
liger+wnn,teadog_full,5.861326e-01,0.955855,0.281586,0.607858,0.584078,0.445244,0.712655,0.984601,0.681645,0.652130


In [55]:
# df_norm = copy.deepcopy(df)
# for metric in metrics:
#     v = [df[method][metric].values[0] for method in methods]
#     v_min = min(v)
#     v_max = max(v)
#     for method in methods:
#         df_norm[method][metric] = (df[method][metric] - v_min) / (v_max - v_min)
# df_norm_cat = pd.concat(df_norm.values(), axis=0)
# df_norm_cat["batch_score"] = (df_norm_cat["iLISI"] + df_norm_cat["graph_conn"] + df_norm_cat["kBET"]) / 3
# df_norm_cat["bio_score"] = (df_norm_cat["NMI"] + df_norm_cat["ARI"] + df_norm_cat["il_score_f1"] + df_norm_cat["cLISI"]) / 4
# df_norm_cat["overall_score"] = 0.4 * df_norm_cat["batch_score"] + 0.6 * df_norm_cat["bio_score"]
# df_norm_cat = df_norm_cat[["iLISI", "graph_conn", "kBET", "batch_score", "NMI", "ARI", "il_score_f1", "cLISI", "bio_score", "overall_score"]]
# df_norm_cat

In [56]:
out_dir = pj("eval", "plot", "data")
utils.mkdir(out_dir, remove_old=False)
if o.mosaic == 0:
    ms = "_sota_"
elif o.sota == 0:
    ms = "_mosaic_"
else:
    ms = "_sota+mosaic_"
df_mean_cat_sorted.to_excel(pj(out_dir, "scib_metrics"+ms+o.tasks[0].split("_")[0]+"_"+o.experiment+"_"+o.init_model+"_sorted.xlsx"))
df_mean_cat.to_excel(pj(out_dir, "scib_metrics"+ms+o.tasks[0].split("_")[0]+"_"+o.experiment+"_"+o.init_model+"_unsorted.xlsx"))